# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Graphlab Create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

This non-commercial license of GraphLab Create for academic use is assigned to saurabhjogalekar@gmail.com and will expire on July 07, 2018.


[INFO] graphlab.cython.cy_server: GraphLab Create v1.10.1 started. Logging: C:\Users\user\AppData\Local\Temp\graphlab_server_1500964585.log.0


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
sales.head()
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.548088     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 1.579090     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

## 6

In [23]:
#model_all.print_rows(num_rows=30, num_columns=5)
#SFrame.print_rows(num_rows=30, num_columns=5)
coeff = model_all.get('coefficients')

for c in coeff:
    if c['value'] > 0:
        print c

{'index': None, 'stderr': None, 'name': '(intercept)', 'value': 274873.0559504957}
{'index': None, 'stderr': None, 'name': 'bathrooms', 'value': 8468.531086910072}
{'index': None, 'stderr': None, 'name': 'sqft_living', 'value': 24.420720982445214}
{'index': None, 'stderr': None, 'name': 'sqft_living_sqrt', 'value': 350.0605533860648}
{'index': None, 'stderr': None, 'name': 'grade', 'value': 842.0680348976282}
{'index': None, 'stderr': None, 'name': 'sqft_above', 'value': 20.024722417091304}


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [24]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [31]:
import numpy as np
RSS = float('inf')
for l1 in np.logspace(1, 7, num=13):
    model_all = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=l1)
    print "done with creating a model for l1 : ", l1
    
    prediction = model_all.predict(validation)
    rss = sum((prediction - validation['price'])**2)
    
    print "calculated RSS..  ", rss
    
    if rss < RSS:
        RSS = rss        
        print "RSS changed for model with l1 : ", l1, "   current min RSS value is : ", RSS
    
    

done with creating a model for l1 :  10.0
calculated RSS..   6.25766285142e+14
RSS changed for model with l1 :  10.0    current min RSS value is :  6.25766285142e+14
done with creating a model for l1 :  31.6227766017
calculated RSS..   6.25766285362e+14
done with creating a model for l1 :  100.0
calculated RSS..   6.25766286058e+14
done with creating a model for l1 :  316.227766017
calculated RSS..   6.25766288257e+14
done with creating a model for l1 :  1000.0
calculated RSS..   6.25766295212e+14
done with creating a model for l1 :  3162.27766017
calculated RSS..   6.25766317206e+14
done with creating a model for l1 :  10000.0
calculated RSS..   6.25766386761e+14
done with creating a model for l1 :  31622.7766017
calculated RSS..   6.25766606749e+14
done with creating a model for l1 :  100000.0
calculated RSS..   6.25767302792e+14
done with creating a model for l1 :  316227.766017
calculated RSS..   6.25769507644e+14
done with creating a model for l1 :  1000000.0
calculated RSS..   6.

*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

**10**

In [57]:
model_all = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=10)
coeff = model_all.get('coefficients')
n=0
for c in coeff:
    if c['value'] > 0:
        #print c
        n=n+1
print n

18


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [36]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [37]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [48]:
#l1_nnz_list = []
#l1_coeff=[]
max_l1 = 0
min_l1 = float('inf')
for l1 in l1_penalty_values:
    model_all = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=l1)
    nnz_coeff = model_all['coefficients']['value'].nnz()
    print "Non zero coefficients for model with l1 : ", l1, " are :", nnz_coeff
    
    if nnz_coeff > max_nonzeros and l1 > max_l1:
        max_l1 = l1
    if nnz_coeff < max_nonzeros and l1 < min_l1:
        min_l1 = l1
        
    #l1_nnz_list.append(nnz_coeff)
    #l1_coeff.append(l1)
'''
Non zero coefficients for model with l1 :  100000000.0  are : 18
Non zero coefficients for model with l1 :  127427498.57  are : 18
Non zero coefficients for model with l1 :  162377673.919  are : 18
Non zero coefficients for model with l1 :  206913808.111  are : 18
Non zero coefficients for model with l1 :  263665089.873  are : 17
Non zero coefficients for model with l1 :  335981828.628  are : 17
Non zero coefficients for model with l1 :  428133239.872  are : 17
Non zero coefficients for model with l1 :  545559478.117  are : 17
Non zero coefficients for model with l1 :  695192796.178  are : 17
Non zero coefficients for model with l1 :  885866790.41  are : 16
Non zero coefficients for model with l1 :  1128837891.68  are : 15
Non zero coefficients for model with l1 :  1438449888.29  are : 15
Non zero coefficients for model with l1 :  1832980710.83  are : 13
Non zero coefficients for model with l1 :  2335721469.09  are : 12
Non zero coefficients for model with l1 :  2976351441.63  are : 10
Non zero coefficients for model with l1 :  3792690190.73  are : 6
Non zero coefficients for model with l1 :  4832930238.57  are : 5
Non zero coefficients for model with l1 :  6158482110.66  are : 3
Non zero coefficients for model with l1 :  7847599703.51  are : 1
Non zero coefficients for model with l1 :  10000000000.0  are : 1
'''
print max_l1
print min_l1

Non zero coefficients for model with l1 :  100000000.0  are : 18
Non zero coefficients for model with l1 :  127427498.57  are : 18
Non zero coefficients for model with l1 :  162377673.919  are : 18
Non zero coefficients for model with l1 :  206913808.111  are : 18
Non zero coefficients for model with l1 :  263665089.873  are : 17
Non zero coefficients for model with l1 :  335981828.628  are : 17
Non zero coefficients for model with l1 :  428133239.872  are : 17
Non zero coefficients for model with l1 :  545559478.117  are : 17
Non zero coefficients for model with l1 :  695192796.178  are : 17
Non zero coefficients for model with l1 :  885866790.41  are : 16
Non zero coefficients for model with l1 :  1128837891.68  are : 15
Non zero coefficients for model with l1 :  1438449888.29  are : 15
Non zero coefficients for model with l1 :  1832980710.83  are : 13
Non zero coefficients for model with l1 :  2335721469.09  are : 12
Non zero coefficients for model with l1 :  2976351441.63  are : 10

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [49]:
l1_penalty_min = max_l1
l1_penalty_max = min_l1

***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

**2976351441.63
3792690190.73**

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [50]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [55]:
RSS = float('inf')
for l1 in l1_penalty_values:
    model_all = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=l1)
    print "done with creating a model for l1 : ", l1
    
    prediction = model_all.predict(validation)
    rss = sum((prediction - validation['price'])**2)
    nnz_coeff = model_all['coefficients']['value'].nnz()
    
    print "calculated RSS :   ", rss, "  number of nonzero coeff : ", nnz_coeff
    
    if rss < RSS and nnz_coeff == max_nonzeros:
        RSS = rss        
        print "RSS changed for model with l1 : ", l1, "   current min RSS value is : ", RSS
        coeff = model_all.get('coefficients')
        for c in coeff:
            if c['value'] > 0:
                print "features with non zero....",c

done with creating a model for l1 :  2976351441.63
calculated RSS :    9.66925692362e+14   number of nonzero coeff :  10
done with creating a model for l1 :  3019316638.95
calculated RSS :    9.74019450085e+14   number of nonzero coeff :  10
done with creating a model for l1 :  3062281836.27
calculated RSS :    9.81188367942e+14   number of nonzero coeff :  10
done with creating a model for l1 :  3105247033.59
calculated RSS :    9.89328342459e+14   number of nonzero coeff :  10
done with creating a model for l1 :  3148212230.92
calculated RSS :    9.98783211266e+14   number of nonzero coeff :  10
done with creating a model for l1 :  3191177428.24
calculated RSS :    1.00847716702e+15   number of nonzero coeff :  10
done with creating a model for l1 :  3234142625.56
calculated RSS :    1.01829878055e+15   number of nonzero coeff :  10
done with creating a model for l1 :  3277107822.88
calculated RSS :    1.02824799221e+15   number of nonzero coeff :  10
done with creating a model for l

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [ ]:
'''
done with creating a model for l1 :  3448968612.16
calculated RSS :    1.04693748875e+15   number of nonzero coeff :  7
RSS changed for model with l1 :  3448968612.16    current min RSS value is :  1.04693748875e+15
'''

